### Chatbot based on Seq2Seq Beam Search + Attention + Reinforcment Learning(Experimental)
- Tensorflow 1.4.0+ is required.
- This is based on [NMT Tutorial](https://github.com/tensorflow/nmt).
- Experiment [notes](https://github.com/higepon/tensorflow_seq2seq_chatbot/wiki).



In [1]:
# Special commands should be located here.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!apt-get -qq install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

!pip -q install git+https://github.com/mrahtz/easy-tf-log#egg=easy-tf-log[tf]
!pip install pushbullet.py
!pip install tweepy pyyaml
!pip install mecab-python3

def auth_google_drive():
  # Generate creds for the Drive FUSE library.
  if not os.path.exists('drive'):
    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}  

def mount_google_drive():
  if not os.path.exists('drive'):
    os.makedirs('drive', exist_ok=True)
    !google-drive-ocamlfuse drive 
    
def kill_docker():
  !kill -9 -1  


Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up gnupg-agent (2.1.15-1ubuntu8.1) ...
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up gnupg (2.1.15-1ubuntu8.1) 

reading /usr/share/mecab/dic/ipadic/Verb.csv ... 130750
reading /usr/share/mecab/dic/ipadic/Noun.place.csv ... 72999
reading /usr/share/mecab/dic/ipadic/Postp.csv ... 146
reading /usr/share/mecab/dic/ipadic/Auxil.csv ... 199
reading /usr/share/mecab/dic/ipadic/Filler.csv ... 19
reading /usr/share/mecab/dic/ipadic/Noun.number.csv ... 42
reading /usr/share/mecab/dic/ipadic/Suffix.csv ... 1393
reading /usr/share/mecab/dic/ipadic/Noun.nai.csv ... 42
reading /usr/share/mecab/dic/ipadic/Conjunction.csv ... 171
reading /usr/share/mecab/dic/ipadic/Noun.org.csv ... 16668
reading /usr/share/mecab/dic/ipadic/Noun.adjv.csv ... 3328
reading /usr/share/mecab/dic/ipadic/Noun.proper.csv ... 27327
emitting double-array: 100% |###########################################| 
reading /usr/share/mecab/dic/ipadic/matrix.def ... 1316x1316
emitting matrix      : 100% |###########################################| 

done!
update-alternatives: using /var/lib/mecab/dic/ipadic to provide /var/lib/mecab/dic/debian (m

emitting double-array: 100% |###########################################| 
reading /usr/share/mecab/dic/ipadic/matrix.def ... 1316x1316
emitting matrix      : 100% |###########################################| 

done!
  easy-tf-log 1.7 does not provide the extra 'tf'
    100% |████████████████████████████████| 204kB 4.6MB/s 
    100% |████████████████████████████████| 286kB 9.5MB/s 
  Running setup.py bdist_wheel for PySocks ... - done
  Stored in directory: /content/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks
    100% |████████████████████████████████| 51kB 2.5MB/s 
  Running setup.py bdist_wheel for mecab-python3 ... - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/4c/07/3a/5f22ccc9f381f3bc01fa023202061cd1e0e9af855292f005dd
Successfully built mecab-python3


In [0]:
import urllib.request
response = urllib.request.urlopen("https://raw.githubusercontent.com/yaroslavvb/memory_util/master/memory_util.py")
open("memory_util.py", "wb").write(response.read())
import memory_util

In [0]:
from __future__ import print_function

import copy as copy
import datetime
import hashlib
import json
import os
import os.path
import filecmp
import random
import re
import shutil
import importlib


import MeCab
import easy_tf_log
import matplotlib.pyplot as plt
import random as random
import numpy as np
import tensorflow as tf
import tweepy
import yaml
from easy_tf_log import tflog
from google.colab import auth
from google.colab import files
import importlib
from pushbullet import Pushbullet
from tensorflow.python.layers import core as layers_core
from tensorflow.python.platform import gfile

# Generate auth tokens for Colab
auth.authenticate_user()


In [0]:
#kill_docker()

In [5]:
auth_google_drive()
mount_google_drive()

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [6]:
import drive.tensorflow_seq2seq_chatbot.lib.chatbot_model as sq

client1
downloading config.yml...
downloaded
1.9.0-rc2
module reloaded9


In [0]:
def reload_modules():
  !fusermount -u drive
  !google-drive-ocamlfuse -cc drive 
  importlib.reload(sq)


In [8]:
if sq.mode == sq.Mode.Test:
    sq.test_distributed_one(enable_attention=False)
    sq.test_distributed_one(enable_attention=True)


==== test_distributed_pattern[ beam] ====
Instructions for updating:
Use the `axis` argument instead
$$$ GPU ENABLED $$$
$$$ GPU ENABLED $$$
Created fresh model.
INFO:tensorflow:Restoring parameters from model/test_distributed/ChatbotModel-100
Inferred replies [6 3 7 4 1]
logits [[-1.2900714  -5.8808107  -0.7343339   4.3347187  -4.059275    1.5587475
   5.474302    0.17898077 -0.9195448 ]
 [-1.6834896  -6.892524   -3.1048987   5.6801476  -1.9099369   0.25509596
   3.9646018   2.7475753  -0.9823034 ]
 [-2.2441032  -2.8392751  -3.6622686   2.0367935   2.1250422   1.1793109
   0.4581913   4.0886135  -1.6223851 ]
 [-1.5222183   2.366472   -2.0782087  -2.616803    4.596033    1.687948
  -2.8720033   2.9211202  -1.306099  ]
 [-0.6432748   6.966592    1.3353962  -6.4133725   4.187848    1.9390206
  -4.048463    0.16939133 -0.9549587 ]]
Inferred replies candidate0 [6 3 7 4 1]
Inferred replies candidate1 [3 3 7 4 1]
sample replies [[6 3 7 4 1]
 [6 3 7 4 1]
 [6 3 7 4 1]]
Expected replies [6 3 7 

In [9]:
tweet_small_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        'batch_size': 6,  # of tweets should be dividable by batch_size
        'encoder_length': 8,
        'decoder_length': 8,
        'num_units': 256,
        'num_layers': 2,
        'vocab_size': 34,
        'embedding_size': 40,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 200,
        'model_path': sq.ModelDirectory.tweet_small.value,
        'learning_rate': 0.05,
        'use_attention': True,
    })

tweet_small_swapped_hparams = copy.deepcopy(
    tweet_small_hparams).override_from_dict(
    {'model_path': sq.ModelDirectory.tweet_small_swapped.value})

if sq.mode == sq.Mode.Test:
    tweets_path = "tweets_small.txt"
    sq.TrainDataGenerator(tweets_path, tweet_small_hparams).remove_generated()
    trainer = sq.Trainer()
    trainer.train_seq2seq(tweet_small_hparams, tweets_path,
                          ["おはようございます。寒いですね。", "さて帰ろう。明日は早い。", "今回もよろしくです。"])
    sq.test_tweets_small_swapped(tweet_small_swapped_hparams)


===== Train Seq2Seq tweets_small.txt ====
{'machine': 'client1', 'batch_size': 6, 'num_units': 256, 'num_layers': 2, 'vocab_size': 34, 'embedding_size': 40, 'learning_rate': 0.05, 'learning_rate_decay': 0.99, 'use_attention': True, 'encoder_length': 8, 'decoder_length': 8, 'max_gradient_norm': 5.0, 'beam_width': 2, 'num_train_steps': 200, 'model_path': 'model/tweet_small'}
downloading tweets_small.txt...
downloaded
generating enc and dec files...
generating vocab file...
loading vocab...
generating id files...
generating padded input file...
generating dec eos/sos files...
done
done
$$$ GPU ENABLED $$$
$$$ GPU ENABLED $$$
$$$ GPU ENABLED $$$
Created fresh model.
....................INFO:tensorflow:Restoring parameters from model/tweet_small/ChatbotModel-21
==== 21 ====
おはようございます。寒いですね。
    [0]ございございますます 
    [1]ございございます  
    [2]ございございますます 
さて帰ろう。明日は早い。
    [0]おございますます 
    [1]ございございますます 
    [2]おございますます 
今回もよろしくです。
    [0]ございますます 
    [1]ございますます 
    [2]おはようますます 
average reply len=9.3

    [0]こちらこそよろしくお願いしします。
    [1]こちらこそよろしくお願いしします。
    [2]こちらこそよろしくお願いお願いします。
average reply len=13.7
validation loss=5.4
...................INFO:tensorflow:Restoring parameters from model/tweet_small/ChatbotModel-141
==== 141 ====
おはようございます。寒いですね。
    [0]おはようございます！！。 
    [1]おはようございます！   
    [2]おはようございます！！。 
さて帰ろう。明日は早い。
    [0]おつかれさまー。気気
    [1]おつかれさまー。気気
    [2]おつかれさまーー気気
今回もよろしくです。
    [0]こちらこそよろしくお願いしします。
    [1]こちらこそよろしくお願いしします。
    [2]こちらこそよろしくお願いしますます。
average reply len=13.3
validation loss=4.2
...................INFO:tensorflow:Restoring parameters from model/tweet_small/ChatbotModel-161
==== 161 ====
おはようございます。寒いですね。
    [0]おはようございます！！。 
    [1]おはようございます！   
    [2]おはようございます！！。 
さて帰ろう。明日は早い。
    [0]おつかれさまー。気気
    [1]おつかれさまー。気気
    [2]おつかれさまー。。気
今回もよろしくです。
    [0]こちらこそよろしくお願いします。。
    [1]こちらこそよろしくお願いします。。
    [2]こちらこそよろしくお願いしますます。
average reply len=13.3
validation loss=3.5
...................INFO:tensorflow:Restoring parameters from model/tweet_small/ChatbotModel-181
==== 181 =

おつかれさまー。気をつけて。
    [0]さて帰ろう。明日 
    [1]さて帰ろう。明日 
    [2]さて帰ろう。です 
こちらこそよろしくお願いします。
    [0]さて帰ろよろしく。です 
    [1]さて帰ろよろしく。です 
    [2]さて帰ろう。です 
average reply len=10.0
validation loss=11.4
...................INFO:tensorflow:Restoring parameters from model/tweet_small_swapped/ChatbotModel-81
==== 81 ====
@higepon おはようございます！
    [0]おはようございますですです 
    [1]おはようございますですです 
    [2]今回ございますですです 
おつかれさまー。気をつけて。
    [0]さて帰ろう明日明日 
    [1]さて帰ろう明日明日 
    [2]さて帰ろよろしく明日明日 
こちらこそよろしくお願いします。
    [0]さて帰ろよろしく。です 
    [1]さて帰ろよろしく。です 
    [2]さてもよろしく。です 
average reply len=12.0
validation loss=9.6
...................INFO:tensorflow:Restoring parameters from model/tweet_small_swapped/ChatbotModel-101
==== 101 ====
@higepon おはようございます！
    [0]おはようございますですです 
    [1]おはようございますですです 
    [2]おはようございますです寒い 
おつかれさまー。気をつけて。
    [0]さて帰ろう明日明日 
    [1]さて帰ろう明日明日 
    [2]さて帰ろう。明日 
こちらこそよろしくお願いします。
    [0]今回もよろしく。です 
    [1]今回もよろしく。です 
    [2]今回帰ろよろしく。です 
average reply len=11.7
validation loss=8.0
...................INFO:tensorflow:

In [0]:
tweet_large_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        # In typical seq2seq chatbot
        # num_layers=3, learning_rate=0.5, batch_size=64, vocab=20000-100000, learning_rate decay is 0.99, which is taken care as default parameter in AdamOptimizer.
        'batch_size': 64,  # of tweets should be dividable by batch_size
        'encoder_length': 28,
        'decoder_length': 28,
        'num_units': 1024,
        'num_layers': 3,
        'vocab_size': 60000,
    # conversations.txt actually has about 70K uniq words.
        'embedding_size': 1024,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 1000000,
        'model_path': sq.ModelDirectory.tweet_large.value,
        'learning_rate': 0.5,
    # For vocab_size 50000, num_layers 3, num_units 1024, tweet_large, starting learning_rate 0.05 works well, change it t0 0.01 at perplexity 800, changed it to 0.005 at 200.
        'learning_rate_decay': 0.99,
        'use_attention': True,
        # testing new restore learning rate and no USERNAME TOKEN
    })

tweet_large_swapped_hparams = copy.deepcopy(
    tweet_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.tweet_large_swapped.value
    })

#Shell.save_model_in_drive(tweet_large_hparams.model_path)

if sq.mode == sq.Mode.TrainSeq2Seq:
    print("train seq2seq")
    sq.test_tweets_large(tweet_large_hparams)
elif sq.mode == sq.Mode.TrainSeq2SeqSwapped:
    print("train seq2seq swapped")
    sq.test_tweets_large_swapped(tweet_large_swapped_hparams)


In [11]:
!ls -Sl model/conversations_large*

model/conversations_large:
total 0

model/conversations_large_backward:
total 0

model/conversations_large_rl:
total 0


In [0]:
  
reload_modules()

conversations_large_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        # In typical seq2seq chatbot
        # num_layers=3, learning_rate=0.5, batch_size=64, vocab=20000-100000, learning_rate decay is 0.99, which is taken care as default parameter in AdamOptimizer.
        'batch_size': 64,  # of tweets should be dividable by batch_size
        'encoder_length': 28,
        'decoder_length': 28,
        'num_units': 1024,
        'num_layers': 3,
        'vocab_size': 60000,
    # conversations.txt actually has about 70K uniq words.
        'embedding_size': 1024,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 0,
        'model_path': sq.ModelDirectory.conversations_large.value,
        'learning_rate': 0.5,
    # For vocab_size 50000, num_layers 3, num_units 1024, tweet_large, starting learning_rate 0.05 works well, change it t0 0.01 at perplexity 800, changed it to 0.005 at 200.
        'learning_rate_decay': 0.99,
        'use_attention': True,

    })

conversations_large_rl_hparams = copy.deepcopy(
    conversations_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.conversations_large_rl.value,
        'num_train_steps': 8000,
        'beam_width': 3,
    })


conversations_large_backward_hparams = copy.deepcopy(
    conversations_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.conversations_large_backward.value,
        'num_train_steps': 0,        
    })


conversations_txt = "conversations_large.txt"
sq.Shell.download_file_if_necessary(conversations_txt)
sq.ConversationTrainDataGenerator().generate(conversations_txt)

with memory_util.capture_stderr() as stderr:
    try:
        trainer =sq.Trainer()
        valid_tweets = ["さて福岡行ってきます！", "誰か飲みに行こう", "熱でてるけど、でもなんか食べなきゃーと思ってアイス買おうとしたの",
              "今日のドラマ面白そう！", "お腹すいたー", "おやすみ～", "おはようございます。寒いですね。",
              "さて帰ろう。明日は早い。", "今回もよろしくです。", "ばいとおわ！"]
        trainer.train_seq2seq(conversations_large_hparams,
                              "conversations_large_seq2seq.txt",
                              valid_tweets, should_clean_saved_model=False)
        trainer.train_seq2seq_swapped(conversations_large_backward_hparams,
                                      "conversations_large_seq2seq.txt",
                                      ["この難にでも応用可能なひどいやつ", "おはようございます。明日はよろしくおねがいします。"], vocab_path="conversations_large_seq2seq_vocab.txt", should_clean_saved_model=False)

        sq.Shell.copy_saved_model(conversations_large_hparams, conversations_large_rl_hparams)
        sq.Trainer().train_rl(conversations_large_rl_hparams,
                                conversations_large_hparams,
                                conversations_large_backward_hparams,
                                "conversations_large_seq2seq.txt",

                                "conversations_large_rl.txt",
                                valid_tweets)
    except Exception as e:
        print(stderr.getvalue())
        raise (e)

!ls - lSh



Clearing cache...done
client1
1.9.0-rc2
module reloaded9
downloading conversations_large.txt...
downloaded
===== Train Seq2Seq conversations_large_seq2seq.txt ====
{'machine': 'client1', 'batch_size': 64, 'num_units': 1024, 'num_layers': 3, 'vocab_size': 60000, 'embedding_size': 1024, 'learning_rate': 0.5, 'learning_rate_decay': 0.99, 'use_attention': True, 'encoder_length': 28, 'decoder_length': 28, 'max_gradient_norm': 5.0, 'beam_width': 2, 'num_train_steps': 0, 'model_path': 'model/conversations_large'}
generating enc and dec files...
generating vocab file...
vocab_len= 391565
loading vocab...
generating id files...
generating padded input file...
generating dec eos/sos files...
done
Copying  drive/seq2seq_data/model/conversations_large/events.out.tfevents.1530186369.67a0b8527e19
Copying  drive/seq2seq_data/model/conversations_large/events.out.tfevents.1530186319.67a0b8527e19
Copying  drive/seq2seq_data/model/conversations_large/events.out.tfevents.1530109115.1cfeca23cb88
Copying  d

In [1]:
# N.B: This would fail if we try to download logs in the previous cell.
# My guess is tflog is somehow locking the log file when running the cell.
sq.Shell.download_logs(conversations_large_rl_hparams.model_path)

NameError: ignored